In [ ]:
import sys

sys.path.append("../")

In [ ]:
# 주식 데이터를 다운로드합니다.

import sqlite3
from pathlib import Path
from typing import Literal

import pandas as pd
import requests
import yfinance as yf


def download_daily_stock(
    ticker: str, save_as: Literal["parquet", "sqlite"] = "parquet"
) -> pd.DataFrame:
    """Download stock data.

    Maximum range, daily. Example) TSLA(Tesla) data is from 2010-06-29 to today.
    Columns : Date, OHLCV, Ticker

    Args:
        ticker (str): ticker.
        save_as (Literal['parquet', 'sqlite'], optional): saving filetype. Defaults to 'parquet'.

    Returns:
        pd.DataFrame: stock dataframe.
    """
    # Download data
    stock = yf.download(ticker, period="max", interval="1d", multi_level_index=False)
    if len(stock) == 0:
        session = requests.Session()
        session.verify = False
        stock = yf.download(
            ticker,
            period="max",
            interval="1d",
            multi_level_index=False,
            session=session,
        )

    # Process data
    stock = stock.reset_index()
    stock["Date"] = pd.to_datetime(stock["Date"])
    stock["Ticker"] = ticker

    # Use Adj Close as Close
    if "Adj Close" in stock.columns:
        stock["Close"] = stock["Adj Close"]
        stock = stock.drop(columns="Adj Close")
    stock = stock.loc[:, ["Date", "Open", "High", "Low", "Close", "Volume", "Ticker"]]

    # Save
    dir_path = Path("data")
    dir_path.mkdir(parents=True, exist_ok=True)

    if save_as == "parquet":
        file_path = dir_path / Path(f"{ticker}.parquet")
        stock.to_parquet(file_path, index=False)
    elif save_as == "sqlite":
        CREATE_TABLE_QUERY = """
        CREATE TABLE IF NOT EXISTS stock (
        date TIMESTAMP,
        open FLOAT,
        high FLOAT,
        low FLOAT,
        close FLOAT,
        volume FLOAT,
        ticker TEXT,
        UNIQUE(date, ticker)
        );
        """
        INSERT_QUERY = """
        INSERT OR IGNORE INTO stock (date, open, high, low, close, volume, ticker) VALUES (?, ?, ?, ?, ?, ?, ?)
        """
        file_path = dir_path / Path("stock.db")
        conn = sqlite3.connect(file_path)
        cursor = conn.cursor()
        cursor.execute(CREATE_TABLE_QUERY)
        values = stock.to_numpy()
        values[:, 0] = stock["Date"].dt.to_pydatetime()
        cursor.executemany(INSERT_QUERY, values)
        conn.commit()
        conn.close()
    return stock


tickers = [
    "SPY",
    "MSFT",
    "AMZN",
    "GOOG",
    "AAPL",
    "META",
    "NVDA",
    "UNH",
    "MA",
    "JNJ",
    "LLY",
    "XOM",
    "V",
    "JPM",
    "BRK-B",
    "PEP",
    "TSLA",
]
for ticker in tickers:
    df = download_daily_stock(ticker, save_as="parquet")
df

In [ ]:
import pandas as pd
from fitted_model import models
from src.bt import *

stocks = []
for file in Path("data").glob("*.parquet"):
    if "SPY" in file.name:
        spy_data = pd.read_parquet(file)
    else:
        stocks.append(pd.read_parquet(file))

In [ ]:
start = "2005-01-01"
end = "2099-11-01"
backtester = Backtester(
    models=models, stocks=stocks, stock_baseline=spy_data, start=start, end=end
)

In [ ]:
p = {
    "7_RSI_period": 2,
    "9_MACDDivision_period_short": 25,
    "9_MACDDivision_period_long": 26,
    "9_MACDDivision_period_signal": 20,
    "9_MACDDivision_use_exponential_moving_average": False,
    "10_BuyCloseOver200MA_multiplier": 0.2625449890445556,
    "11_Buy20Over600MA_multiplier": 0.3855603135509221,
    "12_Buy60Over120MA_multiplier": 0.01,
    "13_Buy120Over200MA_multiplier": 0.2179788842952079,
    "14_BuyBelowLowerBand_multiplier": 0.779702958286791,
    "15_BuyAtLowRSI_right": 36.35446017745976,
    "15_BuyAtLowRSI_multiplier": 1.0,
    "16_BuyNarrowBandGap1_right": 0.1,
    "16_BuyNarrowBandGap1_multiplier": 0.01,
    "17_BuyNarrowBandGap2_right": 2.340943649144741,
    "17_BuyNarrowBandGap2_multiplier": 0.01,
    "18_BuyNarrowBandGap3_right": 1.4673876910061119,
    "18_BuyNarrowBandGap3_multiplier": 1.0,
    "19_MACDGoldenCross_multiplier": 0.41713593502799307,
    "20_MACDGoesUp_right": -7.966768947419718,
    "20_MACDGoesUp_multiplier": 1.0,
    "21_SellAtHighRSI_right": 84.19549872716344,
    "21_SellAtHighRSI_multiplier": 1.0,
    "22_SellAboveUpperBand_multiplier": 0.4937539048229229,
    "23_BuyNarrowBandGap1_right": 1.8117132255034107,
    "23_BuyNarrowBandGap1_multiplier": 0.8223363829080419,
    "24_BuyNarrowBandGap2_right": 0.1,
    "24_BuyNarrowBandGap2_multiplier": 0.7029088317112149,
    "25_BuyNarrowBandGap3_right": 0.8254105363411757,
    "25_BuyNarrowBandGap3_multiplier": 0.01,
    "26_SellCloseUnder200MA_multiplier": 0.7896306855486452,
    "27_Sell20Under600MA_multiplier": 0.01,
    "28_Sell60Under120MA_multiplier": 1.0,
    "29_Sell120Under200MA_multiplier": 0.4533905575568441,
    "0_BollingerBand_period": 20,
    "0_BollingerBand_upper_multiplier": 1.0,
    "0_BollingerBand_lower_multiplier": 1.0,
    "0_BollingerBand_use_exponential_moving_average": False,
    "1_BollingerBand_period": 20,
    "1_BollingerBand_upper_multiplier": 2.0,
    "1_BollingerBand_lower_multiplier": 2.0,
    "1_BollingerBand_use_exponential_moving_average": False,
    "2_BollingerBand_period": 20,
    "2_BollingerBand_upper_multiplier": 3.0,
    "2_BollingerBand_lower_multiplier": 3.0,
    "2_BollingerBand_use_exponential_moving_average": False,
    "3_MA20_period": 20,
    "3_MA20_use_exponential_moving_average": False,
    "4_MA60_period": 60,
    "4_MA60_use_exponential_moving_average": False,
    "5_MA120_period": 120,
    "5_MA120_use_exponential_moving_average": False,
    "6_MA200_period": 200,
    "6_MA200_use_exponential_moving_average": False,
}

In [ ]:
p2 = {
    "0_BollingerBand_lower_multiplier": 1,
    "0_BollingerBand_period": 20,
    "0_BollingerBand_upper_multiplier": 1,
    "0_BollingerBand_use_exponential_moving_average": False,
    "10_BuyCloseOver200MA_multiplier": 5.761711496,
    "11_Buy20Over600MA_multiplier": 1.74889309,
    "12_Buy60Over120MA_multiplier": 3.525587147,
    "13_Buy120Over200MA_multiplier": 5.257117773,
    "14_BuyBelowLowerBand_multiplier": 9.302148531,
    "15_BuyAtLowRSI_multiplier": 10.0,
    "15_BuyAtLowRSI_right": 19.34687661,
    "16_BuyNarrowBandGap1_multiplier": 8.197355052,
    "16_BuyNarrowBandGap1_right": 0.818143261,
    "17_BuyNarrowBandGap2_multiplier": 5.267973974,
    "17_BuyNarrowBandGap2_right": 2.453411177,
    "18_BuyNarrowBandGap3_multiplier": 10.0,
    "18_BuyNarrowBandGap3_right": 2.852508157,
    "19_MACDGoldenCross_multiplier": 1.61784006,
    "1_BollingerBand_lower_multiplier": 2,
    "1_BollingerBand_period": 20,
    "1_BollingerBand_upper_multiplier": 2,
    "1_BollingerBand_use_exponential_moving_average": False,
    "20_MACDGoesUp_multiplier": 2.555281666,
    "20_MACDGoesUp_right": -10.0,
    "21_SellAtHighRSI_multiplier": 0.583772543,
    "21_SellAtHighRSI_right": 100.0,
    "22_SellAboveUpperBand_multiplier": 8.689660748,
    "23_BuyNarrowBandGap1_multiplier": 9.576860056,
    "23_BuyNarrowBandGap1_right": 0.1,
    "24_BuyNarrowBandGap2_multiplier": 4.408108728,
    "24_BuyNarrowBandGap2_right": 1.328117846,
    "25_BuyNarrowBandGap3_multiplier": 9.857276142,
    "25_BuyNarrowBandGap3_right": 0.624092678,
    "26_SellCloseUnder200MA_multiplier": 8.986745906,
    "27_Sell20Under600MA_multiplier": 10.0,
    "28_Sell60Under120MA_multiplier": 6.476415481,
    "29_Sell120Under200MA_multiplier": 3.066642742,
    "2_BollingerBand_lower_multiplier": 3,
    "2_BollingerBand_period": 20,
    "2_BollingerBand_upper_multiplier": 3,
    "2_BollingerBand_use_exponential_moving_average": False,
    "3_MA20_period": 20,
    "3_MA20_use_exponential_moving_average": False,
    "4_MA60_period": 60,
    "4_MA60_use_exponential_moving_average": False,
    "5_MA120_period": 120,
    "5_MA120_use_exponential_moving_average": False,
    "6_MA200_period": 200,
    "6_MA200_use_exponential_moving_average": False,
    "7_RSI_period": 2,
    "9_MACDDivision_period_long": 40,
    "9_MACDDivision_period_short": 25,
    "9_MACDDivision_period_signal": 9,
    "9_MACDDivision_use_exponential_moving_average": True,
}


In [ ]:
pred = backtester.predict(start="20250114", parameters=p)
pred["trade"].loc[pred["trade"]["Quantity"] != 0, :].tail(50)

In [ ]:
pred2 = backtester.predict(start="20250114", parameters=p2)
pred2["trade"].loc[pred2["trade"]["Quantity"] != 0, :].tail(50)

In [ ]:
pred["portfolio"]["New"] = pred2["portfolio"]["Portfolio"]
pred["portfolio"].plot(backend="plotly")